# Fisher forecasting for CMB-S4

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np, scipy as sc, sys, argparse, os
sys.path.append('modules')
import tools

In [3]:
#get the necessary arguments
paramfile='params/params_planck_r_0.0_2015_cosmo_lensed_LSS_JM.txt'
#which_spectra='lensed_scalar' #choices are 'lensed_scalar' and 'unlensed_scalar'
which_spectra='unlensed_scalar' #choices are 'lensed_scalar' and 'unlensed_scalar'
use_ilc_nl = 1
round_results = 0
fsky = 0.57 #fix this to clean patch for now
#desired_param_arr = ['ns', 'neff'] #desired parameter for which we are computing the constraints. set to None if you want to analyse the full fisher matrix
desired_param_arr = None #desired parameter for which we are computing the constraints. set to None if you want to analyse the full fisher matrix


#only used if use_ilc_nl = 0
rms_map_T = 2.
fwhm_arcmins = 1.4

In [4]:
#folders containing inputs
camb_folder = 'data/CMB_spectra_derivatives_for_code_comparison/'
draft_results_folder = 'data/DRAFT_results_20200601/s4like_mask/TT-EE-TE/baseline/'


In [5]:
#get fiducual LCDM power spectra computed using CAMB
print('\tget/read fiducual LCDM power spectra computed using CAMB')
camb_fname = '%s/cmb_spectra_%s_Srini.txt' %(camb_folder, which_spectra.replace('_scalar',''))
if (0):
    cl_camb = np.loadtxt(camb_fname)
    els = cl_camb[:,0]
    cl_dic = {}
    cl_dic['TT'] = cl_camb[:,1]
    cl_dic['EE'] = cl_camb[:,2]
    cl_dic['TE'] = cl_camb[:,3]
camb_fname = camb_fname.replace('.txt', '.npy')
camb_dic = np.load(camb_fname, allow_pickle=1).item()
els = camb_dic['els']
cl_dic = camb_dic['Cl_dic']
if (1):#not add_lensing:
    cl_dic.pop('BB')
    cl_dic.pop('PP')
    cl_dic.pop('Tphi')
    cl_dic.pop('Ephi')

	get/read fiducual LCDM power spectra computed using CAMB


In [6]:
#read derivatives
print('\tget/read derivatives')
camb_deriv_fname = '%s/cmb_spectra_derivs_%s_Srini.npy' %(camb_folder, which_spectra.replace('_scalar',''))
if (0):
    cl_deriv_dic_tmp = np.load(camb_deriv_fname, allow_pickle = 1).item()
    cl_deriv_dic = {}
    param_names = []
    for p in sorted( cl_deriv_dic_tmp ):
        if p == 'ell': continue
        cl_deriv_dic[p]={}
        cl_deriv_dic[p]['TT'] = cl_deriv_dic_tmp[p][0]
        cl_deriv_dic[p]['EE'] = cl_deriv_dic_tmp[p][1]
        cl_deriv_dic[p]['TE'] = cl_deriv_dic_tmp[p][2]
        param_names.append( p )
cl_deriv_dic = np.load(camb_deriv_fname, allow_pickle = 1).item()
param_names = sorted( cl_deriv_dic.keys() )

	get/read derivatives


In [7]:
#get experiment specs
print('\tget experiment specs and nl')
pspectra_to_use = ['TT', 'EE', 'TE']
min_l_temp, max_l_temp = 30, 5000
min_l_pol, max_l_pol = 30, 5000
fix_params = ['Alens', 'mnu', 'ws', 'omk'] #but curretnly nothing to fix as we only have a 6+1(neff) LCDM model
prior_dic = {'tau':0.007}#02}#02} #Planck tau prior
#desired_param = 'neff' #desired parameter for which we are computing the constraints. set to None if you want to analyse the full fisher matrix
include_gal = 0
gal_mask = 3 #only valid if galaxy is included

	get experiment specs and nl


In [8]:
#get nl
print('\tget nl')
if use_ilc_nl:
    if not include_gal:
        nlfile = '%s/S4_ilc_galaxy0_27-39-93-145-225-278_TT-EE-TE_lf2-mf12-hf5_AZ.npy' %(draft_results_folder)
    else:
        nlfile = '%s/S4_ilc_galaxy1_27-39-93-145-225-278_TT-EE-TE_lf2-mf12-hf5_galmask%s_AZ.npy' %(draft_results_folder, gal_mask)
    nl_dic, fsky = tools.get_nldic(nlfile, els)
else:
    rms_map_P = rms_map_T * 1.414
    Bl, nl_TT, nl_PP = tools.fn_get_nl(els, rms_map_T, rms_map_P = rms_map_P, fwhm = fwhm_arcmins)
    nl_dic = {}
    nl_dic['TT'] = nl_TT
    nl_dic['EE'] = nl_PP
    nl_dic['TE'] = np.copy(nl_PP)*0.

	get nl


In [9]:
#get delta_cl
print('\tget delta Cl')
delta_cl_dic = tools.fn_delta_Cl(els, cl_dic, nl_dic, fsky)


	get delta Cl


In [10]:
#get Fisher / COV matrices
print('\tget fisher')
F_mat = tools.get_fisher_mat(els, cl_deriv_dic, delta_cl_dic, param_names, pspectra_to_use = pspectra_to_use,\
            min_l_temp = min_l_temp, max_l_temp = max_l_temp, min_l_pol = min_l_pol, max_l_pol = max_l_pol)
#print(F_mat)


	get fisher


In [11]:
#fix params
print('\tfixing paramaters, if need be')
F_mat, param_names = tools.fn_fix_params(F_mat, param_names, fix_params)
param_names = np.asarray(param_names)
#print(param_names); sys.exit()


	fixing paramaters, if need be


In [12]:
#add prior
print('\tadding prior')
F_mat = tools.fn_add_prior(F_mat, param_names, prior_dic)


	adding prior


In [13]:
#get cov matrix now
print('\tget covariance matrix')
#Cov_mat = sc.linalg.pinv2(F_mat) #made sure that COV_mat_l * Cinv_l ~= I
Cov_mat = np.linalg.inv(F_mat) #made sure that COV_mat_l * Cinv_l ~= I


	get covariance matrix


In [14]:
#extract sigma(neff)
if desired_param_arr is None:
    desired_param_arr = param_names
for desired_param in desired_param_arr:
    print('\textract sigma(%s)' %(desired_param))
    pind = np.where(param_names == desired_param)[0][0]
    pcntr1, pcntr2 = pind, pind
    cov_inds_to_extract = [(pcntr1, pcntr1), (pcntr1, pcntr2), (pcntr2, pcntr1), (pcntr2, pcntr2)]
    cov_extract = np.asarray( [Cov_mat[ii] for ii in cov_inds_to_extract] ).reshape((2,2))
    sigma = cov_extract[0,0]**0.5
    if round_results:
        sigma = round(sigma, 4)
    opline = '\t\simga(%s) = %g using observables=%s; fsky=%s; power spectra=%s' %(desired_param, sigma, str(pspectra_to_use), fsky, which_spectra)
    print('%s\n' %(opline))

    ############################################################################################################
    if not use_ilc_nl: #record sigma in a text file for different white noise levels
        opfolder = 'results/'
        if round_results:
            opfolder = '%s/rounding_0p4' %(opfolder)
        else:
            opfolder = '%s/no_rounding' %(opfolder)
        if not os.path.exists(opfolder): os.system('mkdir -p %s' %(opfolder))
        opfname = '%s/%s_fsky%.2f_fwhm%.2fam.txt' %(opfolder, desired_param, fsky, fwhm_arcmins)
        if os.path.exists(opfname):
            opf = open(opfname, 'a')
        else:
            opf = open(opfname, 'w')
            opline = '#noise \sigma(%s)' %(desired_param)
            opf.writelines('%s\n' %(opline))

        opline = '%.3f %g' %(rms_map_T, sigma)
        opf.writelines('%s\n' %(opline))
        opf.close()


	extract sigma(As)
	\simga(As) = 3.08955e-11 using observables=['TT', 'EE', 'TE']; fsky=0.77325439453125; power spectra=unlensed_scalar

	extract sigma(neff)
	\simga(neff) = 0.0272404 using observables=['TT', 'EE', 'TE']; fsky=0.77325439453125; power spectra=unlensed_scalar

	extract sigma(ns)
	\simga(ns) = 0.00259278 using observables=['TT', 'EE', 'TE']; fsky=0.77325439453125; power spectra=unlensed_scalar

	extract sigma(ombh2)
	\simga(ombh2) = 3.59466e-05 using observables=['TT', 'EE', 'TE']; fsky=0.77325439453125; power spectra=unlensed_scalar

	extract sigma(omch2)
	\simga(omch2) = 0.000657076 using observables=['TT', 'EE', 'TE']; fsky=0.77325439453125; power spectra=unlensed_scalar

	extract sigma(tau)
	\simga(tau) = 0.00698175 using observables=['TT', 'EE', 'TE']; fsky=0.77325439453125; power spectra=unlensed_scalar

	extract sigma(thetastar)
	\simga(thetastar) = 6.38439e-07 using observables=['TT', 'EE', 'TE']; fsky=0.77325439453125; power spectra=unlensed_scalar



In [18]:
logline = '\tdebugging Fisher matrix for individual parameter constraints'; print(logline)
F_mat_for_debug = np.copy(F_mat)
F_mat_for_debug_diag = np.diag(F_mat_for_debug)
individual_constraints = np.sqrt(1./F_mat_for_debug_diag)
for pcntr, curr_param in enumerate( param_names ):
    sigma = individual_constraints[pcntr]
    opline = '\t\t\sigma(%s) = %g using %s; fsky = %s; power spectra = %s' %(curr_param, sigma, str(pspectra_to_use), fsky, which_spectra)
    print(opline)
sys.exit()


	debugging Fisher matrix for individual parameter constraints
		\sigma(As) = 9.12364e-13 using ['TT', 'EE', 'TE']; fsky = 0.77325439453125; power spectra = unlensed_scalar
		\sigma(neff) = 0.00329299 using ['TT', 'EE', 'TE']; fsky = 0.77325439453125; power spectra = unlensed_scalar
		\sigma(ns) = 0.000365992 using ['TT', 'EE', 'TE']; fsky = 0.77325439453125; power spectra = unlensed_scalar
		\sigma(ombh2) = 1.34841e-05 using ['TT', 'EE', 'TE']; fsky = 0.77325439453125; power spectra = unlensed_scalar
		\sigma(omch2) = 0.000263622 using ['TT', 'EE', 'TE']; fsky = 0.77325439453125; power spectra = unlensed_scalar
		\sigma(tau) = 0.000207661 using ['TT', 'EE', 'TE']; fsky = 0.77325439453125; power spectra = unlensed_scalar
		\sigma(thetastar) = 3.76613e-07 using ['TT', 'EE', 'TE']; fsky = 0.77325439453125; power spectra = unlensed_scalar


SystemExit: 